# Import Statements and Pandas Settings

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # heatmap

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# stop warnings about future deprecation during KNN
import warnings
warnings.filterwarnings('ignore')

# allows us to see all columns
pd.set_option("display.max_columns", None) 

## Reading in Data
We look at the first rows of the dataframe to make sure that our data is read in correctly. In addition, we look at the information Pandas tells us about the dataframe, including the column names and object types, which give us hints for data cleaning.

In [2]:
cars_data = pd.read_csv('../input/carsforsale/cars_raw.csv')
cars_data.head(5)

In [3]:
cars_data.info()

# 1 Data Cleaning
## 1.1 Used/New
We begin with the column "Used/New". If we look at the unique values in this column, we can see that any car not marked 'Used' is marked with a car brand and 'Certified'. We can standardize all of these values by keeping 'Used' entries unchanged while making anything not marked 'Used' 'Certified'. At the end, we are left with only those two unique values. 


In [4]:
print(cars_data['Used/New'].unique())

In [5]:
cars_data['Used/New'] = cars_data['Used/New'].map({'Used': 'Used'}).fillna('Certified')
print(cars_data['Used/New'].unique())

Since we only have two categories, we can create a new column marked "Used", with a boolean value that denotes if the car is used or not. 

In [6]:
cars_data['Used'] = cars_data['Used/New'] == 'Used'
cars_data['Used']

Note that the values displayed are all true. This is because most cars in this dataset are marked as used:

In [7]:
print('There are', cars_data['Used'].sum(), 'used cars in the dataset at this stage of cleaning.')
print('In contrast, there are only', len(cars_data['Used']) - cars_data['Used'].sum(), 'new cars in the dataset at the time.')

## 1.2 Price
As the .info() told us, the Price column is currently a list of objects. Ideally, we would want this to be a list of integers instead. We remove all rows that do not have a price, and then remove any dollar sign or comma in the Price column. Finally, we convert them to the int type. This converts the Price into a usable list of integers.

In [8]:
cars_data = cars_data[cars_data['Price'] != 'Not Priced']
cars_data["Price"] = cars_data["Price"].replace("[$,]", "", regex=True).astype(int)
cars_data.head()

## 1.3 Drivetrain
Next, we clean the Drivetrain column. All rows with a dash for Drivetrain are removed. The dataset contains multiple ways to describe front-wheel, four-wheel, all-wheel, and rear-wheel drive. The data is grouped into the four categories FWD, 4WD, AWD, and RWD, and the data is made categorical to reflect this.

In [9]:
print(cars_data['Drivetrain'].unique())

In [10]:
drive_dict = {'Front-wheel Drive': 'FWD', 'Front Wheel Drive': 'FWD',
             'Four-wheel Drive': '4WD', 'All-wheel Drive': 'AWD',
             'Rear-wheel Drive': 'RWD'}
cars_data = cars_data[cars_data['Drivetrain'] != '–']
cars_data['Drivetrain'] = cars_data['Drivetrain'].copy().map(drive_dict).fillna(cars_data['Drivetrain']).astype('category')
print(cars_data['Drivetrain'].unique())

## 1.4 Fuel
Similar code is ran to the Drivetrain cleaning to ensure that the FuelType column fits into the categories "Gasoline", "FlexFuel", "Electric", "Hybrid", and "Diesel". 

In [11]:
cars_data['FuelType'].unique()

In [12]:
fuel_dict = {'Gasoline Fuel': 'Gasoline', 'Electric Fuel System': 'Gasoline',
             'E85 Flex Fuel': 'FlexFuel', 'Flex Fuel Capability': 'FlexFuel', 'Flexible Fuel': 'FlexFuel',
             'Plug-In Electric/Gas': 'Hybrid', 'Gasoline/Mild Electric Hybrid': 'Hybrid',
             'Diesel Fuel': 'Diesel'}
cars_data = cars_data[cars_data['FuelType'] != '–']

# .copy() is needed due to some infernal copy warning 
# it sent me on a wild goose chase for an hour
cars_data['FuelType'] = cars_data['FuelType'].copy().map(fuel_dict).fillna(cars_data['FuelType']).astype('category')

print(cars_data['FuelType'].unique())

## 1.5 MinMPG and MaxMPG
The MinMPG and MaxMPG data has some 0s. This is not feasible for cars, so rows containing these are removed. In addition, an "averageMPG" column is created that averages MinMPG and MaxMPG for the car.

In [13]:
print(np.sort(cars_data['MaxMPG'].unique()))
print(np.sort(cars_data['MinMPG'].unique()))

In [14]:
cars_data = cars_data[cars_data['MaxMPG'] != 0]
cars_data = cars_data[cars_data['MinMPG'] != 0]
print(np.sort(cars_data['MaxMPG'].unique()))
print(np.sort(cars_data['MinMPG'].unique()))

In [15]:
cars_data['AverageMPG'] = (cars_data['MinMPG'] + cars_data['MaxMPG']) / 2
cars_data['AverageMPG']


We have no such issues with Mileage:

In [16]:
np.sort(cars_data['Mileage'].unique())

## 1.6 DealType
The DealType column should only have the three values "Great", "Good", and "Fair". However, there are NaN values, so those must be removed

In [17]:
print(cars_data['DealType'].unique())

In [18]:
ratings = ['Great', 'Good', 'Fair']
cars_data = cars_data[cars_data['DealType'].isin(ratings)]
print(cars_data['DealType'].unique())

# 2 Analysis

## 2.1 Correlations

A basic heatmap is used to compare numerical values in the dataset and see if any variables are strongly correlated. As expected, the ConsumerRating variable is strongly correlated to the various ratings for certain aspects of the car (comfort, interior design, etc.). 

Special attention is given to the correlations of variables with Price. The year of the car model also has a decently strong correlation to the price, which again makes sense as newer cars tend to be more expensive than older cars. Interestingly, the consumer rating and consumer reviews are both negatively correlated to price, so better reviews lead to a lower price. This might be due to customers feeling like they get a better deal for what they paid, given how the Value for Money rating is also negatively correlated to price. Finally, a higher mileage leads to a lower price, which again is expected due to the wear-and-tear of cars.


In [19]:
fig, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(cars_data.corr(), square=True, vmax = 0.8)

## 2.2 Setting up the Classification Model
The goal is to design a model that might be able to predict the classification of a deal ("Great", "Good", "Fair") by taking in certain variables.

In [20]:
variables = ['Year', 'Used', 'ConsumerRating', 'SellerRating', 'Price',
             'ComfortRating', 'InteriorDesignRating', 'PerformanceRating',
             'ValueForMoneyRating','ExteriorStylingRating', 'ReliabilityRating',
             'Drivetrain', 'AverageMPG', 'FuelType', 'Mileage', 'DealType']

cars_model = cars_data[variables]

In [21]:
cars_model.head()

The categorical data is now one-hot encoded, and the numerical data is scaled down so that they are in the same range. At the end, both are combined into the same dataframe.

In [22]:
categorical = ['Used', 'Drivetrain', 'FuelType']
numerical = ['Year', 'ConsumerRating', 'SellerRating', 'Price',
             'ComfortRating', 'InteriorDesignRating', 'PerformanceRating',
             'ValueForMoneyRating','ExteriorStylingRating', 'ReliabilityRating',
             'AverageMPG', 'Mileage']

ohe = OneHotEncoder(sparse=False)
cars_model_cat = pd.DataFrame(ohe.fit_transform(cars_model[categorical]))
scale = StandardScaler()
cars_model_num = pd.DataFrame(scale.fit_transform(cars_model[numerical]))
cars_model_num.columns = numerical
cars_concat = pd.concat([cars_model_cat.reset_index(drop=True), cars_model_num.reset_index(drop=True)], axis=1)
cars_concat

DealType is made into its own series, deal_type.

In [23]:
deal_type = cars_model['DealType'].reset_index(drop=True)
deal_type

## 2.3: Implementing K-nearest-neighbors
The data is split into train-test splits, so the trained model isn't hyper-specified for the specific data it is given, and can perform on generalized data. After making sure that the dimensions of the data are correct, we try a range of k-values from 1 to 100, plot the model accuracy on the test split, and choose a value of k that performed well.

In [24]:
x_train, x_test, y_train, y_test = train_test_split(cars_concat, deal_type, test_size = 0.3, random_state = 1)


In [25]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


In [26]:
k_range = range(1, 100)
scores = {}
scores_list = []
for k in k_range:
    if k % 10 == 0:
        print(k) # lets me know if this works or not
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    scores_list.append(metrics.accuracy_score(y_test, y_pred))

plt.plot(k_range, scores_list)
plt.xlabel('K used')
plt.ylabel('Accuracy on Test Split')

## 2.4 Evaluation
Our model scores around 61% when the value of k is around 20-40. I believe the spike at k = 1 is unreliable due to the relatively low-scoring area around it. Therefore, I pick a value of k = 30 for the final model.

In [27]:
knn = KNeighborsClassifier(n_neighbors = 30)
knn.fit(cars_concat, deal_type)

y_predict = knn.predict(cars_concat)

from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(knn, cars_concat, deal_type)
#from sklearn.metrics import confusion_matrix
#print(confusion_matrix(deal_type, y_predict))